# LiteLLM Quickstart

In this quickstart you will learn how to use LiteLLM as a feedback function provider.

[LiteLLM](https://github.com/BerriAI/litellm) is a consistent way to access 100+ LLMs such as those from OpenAI, HuggingFace, Anthropic, and Cohere. Using LiteLLM dramatically expands the model availability for feedback functions. Please be cautious in trusting the results of evaluations from models that have not yet been tested.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/litellm_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need an OpenAI key.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."

### Import from LangChain and TruLens

In [ ]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru, LiteLLM
tru = Tru()
tru.reset_database()


# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

### Send your first request

In [ ]:
prompt_input = 'What is a good name for a store that sells colorful socks?'

In [ ]:
llm_response = chain(prompt_input)

display(llm_response)

## Initialize Feedback Function(s)

In [ ]:
# Initialize LiteLLM-based feedback function collection class:
litellm = LiteLLM(model_engine="gpt-3.5-turbo")

# Define a relevance function using LiteLLM
relevance = Feedback(litellm.relevance_with_cot_reasons).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

## Instrument chain for logging with TruLens

In [ ]:
tru_recorder = TruChain(chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[relevance])

In [ ]:
with tru_recorder as recording:
    llm_response = chain(prompt_input)

display(llm_response)

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0]

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all